<a href="https://cognitiveclass.ai"><img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DL0101EN-SkillsNetwork/images/IDSN-logo.png" width="400"> </a>

# Convolutional Neural Networks with Keras

Estimated time needed **30** mins


In this lab, we will learn how to use the Keras library to build convolutional neural networks. We will also use the popular MNIST dataset and we will compare our results to using a conventional neural network.


## Objectives for this Notebook    
* How to use the Keras library to build convolutional neural networks
* Convolutional neural network with one set of convolutional and pooling layers
* Convolutional neural network with two sets of convolutional and pooling layers



## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
      
1. <a href="#Import-Keras-and-Packages">Import Keras and Packages</a>   
2. <a href="#Convolutional-Neural-Network-with-One-Set-of-Convolutional-and-Pooling-Layers">Convolutional Neural Network with One Set of Convolutional and Pooling Layers</a>  
3. <a href="#Convolutional-Neural-Network-with-Two-Sets-of-Convolutional-and-Pooling-Layers">Convolutional Neural Network with Two Sets of Convolutional and Pooling Layers</a>  

</font>
</div>


### Install the necessary libraries


Let's start by installing the keras libraries and the packages that we would need to build a neural network.


In [1]:
# All Libraries required for this lab are listed below. The libraries need to be installed on Skills Network Labs. 
# If you run this notebook on a different environment, e.g. your desktop, you may want to install these.
!pip install numpy==2.0.2
!pip install pandas==2.2.2
!pip install tensorflow_cpu==2.18.0
!pip install matplotlib==3.9.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 120.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 105.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.3/230.3 MB 40.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 48.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 119.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5

#### Suppress the tensorflow warning messages
We use the following code to  suppress the warning messages due to use of CPU architechture for tensoflow.
You may want to **comment out** these lines if you are using the GPU architechture


In [2]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

## Import Keras and Packages


In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input
from keras.utils import to_categorical

When working with convolutional neural networks in particular, we will need additional packages.


In [4]:
from keras.layers import Conv2D # to add convolutional layers
from keras.layers import MaxPooling2D # to add pooling layers
from keras.layers import Flatten # to flatten data for fully connected layers

## Convolutional Neural Network with One Set of Convolutional and Pooling Layers


In [5]:
# import data
from keras.datasets import mnist

# load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# reshape to be [samples][pixels][width][height]
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32')

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Let's normalize the pixel values to be between 0 and 1


In [6]:
X_train = X_train / 255 # normalize training data
X_test = X_test / 255 # normalize test data

Next, let's convert the target variable into binary categories


In [7]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

num_classes = y_test.shape[1] # number of categories

Next, let's define a function that creates our model. Let's start with one set of convolutional and pooling layers.


In [8]:
def convolutional_model():
    
    # create model
    model = Sequential()
    model.add(Input(shape=(28, 28, 1)))
    model.add(Conv2D(16, (5, 5), strides=(1, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    # compile model
    model.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=['accuracy'])
    return model

Finally, let's call the function to create the model, and then let's train it and evaluate it.


In [9]:
# build the model
model = convolutional_model()

# fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200, verbose=2)

# evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: {} \n Error: {}".format(scores[1], 100-scores[1]*100))

Epoch 1/10
300/300 - 37s - 124ms/step - accuracy: 0.9145 - loss: 0.3095 - val_accuracy: 0.9685 - val_loss: 0.1112
Epoch 2/10
300/300 - 36s - 121ms/step - accuracy: 0.9736 - loss: 0.0924 - val_accuracy: 0.9808 - val_loss: 0.0652
Epoch 3/10
300/300 - 36s - 120ms/step - accuracy: 0.9814 - loss: 0.0621 - val_accuracy: 0.9820 - val_loss: 0.0583
Epoch 4/10
300/300 - 35s - 118ms/step - accuracy: 0.9849 - loss: 0.0498 - val_accuracy: 0.9849 - val_loss: 0.0478
Epoch 5/10
300/300 - 36s - 118ms/step - accuracy: 0.9879 - loss: 0.0404 - val_accuracy: 0.9841 - val_loss: 0.0445
Epoch 6/10
300/300 - 36s - 119ms/step - accuracy: 0.9900 - loss: 0.0328 - val_accuracy: 0.9859 - val_loss: 0.0446
Epoch 7/10
300/300 - 35s - 118ms/step - accuracy: 0.9917 - loss: 0.0279 - val_accuracy: 0.9856 - val_loss: 0.0445
Epoch 8/10
300/300 - 35s - 118ms/step - accuracy: 0.9930 - loss: 0.0229 - val_accuracy: 0.9869 - val_loss: 0.0395
Epoch 9/10
300/300 - 35s - 118ms/step - accuracy: 0.9940 - loss: 0.0196 - val_accuracy: 

------------------------------------------


## Convolutional Neural Network with Two Sets of Convolutional and Pooling Layers


Let's redefine our convolutional model so that it has two convolutional and pooling layers instead of just one layer of each.


In [10]:
def convolutional_model():
    
    # create model
    model = Sequential()
    model.add(Input(shape=(28, 28, 1)))
    model.add(Conv2D(16, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    model.add(Conv2D(8, (2, 2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    # Compile model
    model.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=['accuracy'])
    return model

Now, let's call the function to create our new convolutional neural network, and then let's train it and evaluate it.


In [11]:
# build the model
model = convolutional_model()

# fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200, verbose=2)

# evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: {} \n Error: {}".format(scores[1], 100-scores[1]*100))

Epoch 1/10
300/300 - 40s - 134ms/step - accuracy: 0.8714 - loss: 0.4483 - val_accuracy: 0.9612 - val_loss: 0.1300
Epoch 2/10
300/300 - 37s - 125ms/step - accuracy: 0.9658 - loss: 0.1140 - val_accuracy: 0.9755 - val_loss: 0.0831
Epoch 3/10
300/300 - 38s - 125ms/step - accuracy: 0.9757 - loss: 0.0822 - val_accuracy: 0.9796 - val_loss: 0.0678
Epoch 4/10
300/300 - 38s - 126ms/step - accuracy: 0.9797 - loss: 0.0663 - val_accuracy: 0.9836 - val_loss: 0.0501
Epoch 5/10
300/300 - 38s - 126ms/step - accuracy: 0.9824 - loss: 0.0578 - val_accuracy: 0.9847 - val_loss: 0.0489
Epoch 6/10
300/300 - 38s - 126ms/step - accuracy: 0.9847 - loss: 0.0513 - val_accuracy: 0.9861 - val_loss: 0.0435
Epoch 7/10
300/300 - 38s - 125ms/step - accuracy: 0.9859 - loss: 0.0453 - val_accuracy: 0.9874 - val_loss: 0.0368
Epoch 8/10
300/300 - 38s - 125ms/step - accuracy: 0.9876 - loss: 0.0400 - val_accuracy: 0.9873 - val_loss: 0.0405
Epoch 9/10
300/300 - 37s - 125ms/step - accuracy: 0.9891 - loss: 0.0356 - val_accuracy: 

<h3>Practice Exercise 1</h3>


Let's see how batch size affects the time required and accuracy of the model training. 
For this, you can try to change batch_size to 1024 and check it's effect on accuracy


In [12]:
# Write your answer here
# build the model
model = convolutional_model()

# fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=1024, verbose=2)

# evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: {} \n Error: {}".format(scores[1], 100-scores[1]*100))


Epoch 1/10
59/59 - 38s - 637ms/step - accuracy: 0.6600 - loss: 1.2447 - val_accuracy: 0.8820 - val_loss: 0.3971
Epoch 2/10
59/59 - 36s - 614ms/step - accuracy: 0.9094 - loss: 0.2985 - val_accuracy: 0.9389 - val_loss: 0.2087
Epoch 3/10
59/59 - 35s - 591ms/step - accuracy: 0.9454 - loss: 0.1833 - val_accuracy: 0.9596 - val_loss: 0.1392
Epoch 4/10
59/59 - 35s - 600ms/step - accuracy: 0.9613 - loss: 0.1339 - val_accuracy: 0.9658 - val_loss: 0.1106
Epoch 5/10
59/59 - 36s - 604ms/step - accuracy: 0.9674 - loss: 0.1098 - val_accuracy: 0.9722 - val_loss: 0.0927
Epoch 6/10
59/59 - 35s - 597ms/step - accuracy: 0.9717 - loss: 0.0942 - val_accuracy: 0.9763 - val_loss: 0.0781
Epoch 7/10
59/59 - 36s - 606ms/step - accuracy: 0.9749 - loss: 0.0830 - val_accuracy: 0.9764 - val_loss: 0.0782
Epoch 8/10
59/59 - 41s - 701ms/step - accuracy: 0.9780 - loss: 0.0746 - val_accuracy: 0.9814 - val_loss: 0.0614
Epoch 9/10
59/59 - 36s - 606ms/step - accuracy: 0.9796 - loss: 0.0684 - val_accuracy: 0.9822 - val_loss:

Double-click <b>here</b> for the solution.

<!-- Your answer is below:
# build the model
model = convolutional_model()

# fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=1024, verbose=2)

# evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: {} \n Error: {}".format(scores[1], 100-scores[1]*100))
-->


<h3>Practice Exercise 2</h3>


Now, let's see how number of epochs  affect the time required and accuracy of the model training. 
For this, you can keep the batch_size=1024 and epochs=25 and check it's effect on accuracy


In [13]:
# Write your answer here
# build the model
model = convolutional_model()

# fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=25, batch_size=1024, verbose=2)

# evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: {} \n Error: {}".format(scores[1], 100-scores[1]*100))


Epoch 1/25
59/59 - 38s - 650ms/step - accuracy: 0.6645 - loss: 1.1945 - val_accuracy: 0.8800 - val_loss: 0.4114
Epoch 2/25
59/59 - 35s - 594ms/step - accuracy: 0.9046 - loss: 0.3232 - val_accuracy: 0.9296 - val_loss: 0.2408
Epoch 3/25
59/59 - 35s - 594ms/step - accuracy: 0.9359 - loss: 0.2177 - val_accuracy: 0.9521 - val_loss: 0.1668
Epoch 4/25
59/59 - 35s - 595ms/step - accuracy: 0.9520 - loss: 0.1623 - val_accuracy: 0.9603 - val_loss: 0.1310
Epoch 5/25
59/59 - 36s - 602ms/step - accuracy: 0.9612 - loss: 0.1303 - val_accuracy: 0.9688 - val_loss: 0.1039
Epoch 6/25
59/59 - 36s - 604ms/step - accuracy: 0.9678 - loss: 0.1095 - val_accuracy: 0.9707 - val_loss: 0.0948
Epoch 7/25
59/59 - 36s - 615ms/step - accuracy: 0.9706 - loss: 0.0983 - val_accuracy: 0.9738 - val_loss: 0.0852
Epoch 8/25
59/59 - 36s - 605ms/step - accuracy: 0.9730 - loss: 0.0884 - val_accuracy: 0.9759 - val_loss: 0.0758
Epoch 9/25
59/59 - 35s - 596ms/step - accuracy: 0.9758 - loss: 0.0812 - val_accuracy: 0.9763 - val_loss:

Double-click <b>here</b> for the solution.

<!-- Your answer is below:
# build the model
model = convolutional_model()

# fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=25, batch_size=1024, verbose=2)

# evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: {} \n Error: {}".format(scores[1], 100-scores[1]*100))


    -->


### Thank you for completing this lab!

This notebook was created by [Alex Aklson](https://www.linkedin.com/in/aklson/). I hope you found this lab interesting and educational. Feel free to contact me if you have any questions!


<!--
## Change Log

|  Date (YYYY-MM-DD) |  Version | Changed By  |  Change Description |
|---|---|---|---|
| 2024-11-20  | 3.0  | Aman  |  Updated the library versions to current |
| 2020-09-21  | 2.0  | Srishti  |  Migrated Lab to Markdown and added to course repo in GitLab |



<hr>

## <h3 align="center"> © IBM Corporation. All rights reserved. <h3/>


## <h3 align="center"> &#169; IBM Corporation. All rights reserved. <h3/>

